# viz

> vizualization routines


**NOTE:** Lazy imports throughout

In [ ]:
#| default_exp viz

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
import torch
import numpy as np
import wandb
import gc

## UMAP

In [ ]:
#| export
def cpu_umap_project(embeddings, n_components=3, n_neighbors=15, min_dist=0.1, random_state=42):
    "Project embeddings to n_components dimensions via UMAP (on CPU)"
    import umap
    if isinstance(embeddings, torch.Tensor): embeddings = embeddings.detach().cpu().numpy()
    reducer = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, random_state=random_state)
    return reducer.fit_transform(embeddings)

In [ ]:
#| export
def cuml_umap_project(embeddings, n_components=3, n_neighbors=15, min_dist=0.1, random_state=42):
    "Project embeddings to n_components dimensions via cuML UMAP (GPU)"
    from cuml import UMAP
    import cupy as cp
    if isinstance(embeddings, torch.Tensor): embeddings = cp.from_dlpack(embeddings.detach())
    reducer = UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, random_state=random_state)
    coords = reducer.fit_transform(embeddings)
    del reducer
    return cp.asnumpy(coords)  # back to numpy for plotly

In [ ]:
#| export
def umap_project(embeddings, **kwargs): 
    "Calls one of two preceding UMAP routines based on device availability."
    try:
        coords = cuml_umap_project(embeddings, **kwargs)
    except torch.cuda.OutOfMemoryError:
        torch.cuda.empty_cache()
        coords = cpu_umap_project(embeddings, **kwargs)
    return coords

## PCA

In [ ]:
#| export
def cuml_pca_project(embeddings, n_components=3):
    "Project embeddings to n_components dimensions via cuML PCA (GPU)"
    from cuml import PCA
    import cupy as cp
    if isinstance(embeddings, torch.Tensor): embeddings = cp.from_dlpack(embeddings.detach())
    coords = PCA(n_components=n_components).fit_transform(embeddings)
    return cp.asnumpy(coords)

In [ ]:
#| export
def cpu_pca_project(embeddings, n_components=3):
    "Project embeddings to n_components dimensions via sklearn PCA (CPU)"
    from sklearn.decomposition import PCA
    if isinstance(embeddings, torch.Tensor): embeddings = embeddings.detach().cpu().numpy()
    return PCA(n_components=n_components).fit_transform(embeddings)

In [ ]:
#| export
def pca_project(embeddings, **kwargs):
    "Calls GPU or CPU PCA based on availability"
    try:
        return cuml_pca_project(embeddings, **kwargs)
    except:
        return cpu_pca_project(embeddings, **kwargs)

## 3D Plotly Scatterplots

In [ ]:
#| export
def plot_embeddings_3d(coords, num_tokens, color_by='pairs', file_idx=None, title='Embeddings', debug=False):
    "3D scatter plot of embeddings. color_by: 'none', 'file', or 'pair'"
    import plotly.graph_objects as go
    n = len(coords)
    if debug: print(" plot_embeddings_3d: n =",n)
    
    if color_by == 'none':     colors = ['blue'] * n
    elif color_by == 'file':   colors = file_idx.tolist() if file_idx is not None else ['blue'] * n
    elif color_by == 'pairs':
        n_pairs = n // 2
        pair_colors = [f'rgb({np.random.randint(0,256)},{np.random.randint(0,256)},{np.random.randint(0,256)})' for _ in range(n_pairs)]
        colors = [pair_colors[i % n_pairs] for i in range(n)]
    else: raise ValueError(f"Unknown color_by: {color_by}")

    hover_text = [f"fileid: {int(fid)}" for fid in file_idx] if file_idx is not None else None
    
    fig = go.Figure(data=[go.Scatter3d(
        x=coords[:,0], y=coords[:,1], z=coords[:,2],
        mode='markers', 
        marker=dict(size=4, color=colors, colorscale='Viridis' if color_by != 'pairs' else None, opacity=0.8),
        hovertext=hover_text,
        hoverinfo='text' if hover_text else 'x+y+z'
    )])
    title = title + f', n={n}'
    fig.update_layout(title=title, margin=dict(l=0, r=0, b=0, t=30))
    return fig


## Main Routine 

Calls the preceding routines

In [ ]:
#| export
def _make_emb_viz(zs, num_tokens, epoch=-1, title='Embeddings', do_umap=True, file_idx=None):
    "visualize embeddings, projected"
    fig = None
    if do_umap:
        coords = umap_project(zs)
        umap_fig = plot_embeddings_3d(coords, num_tokens, title=title+f' (UMAP), epoch {epoch}', file_idx=file_idx)
    if torch.cuda.is_available(): torch.cuda.synchronize() # cleanup before PCA or else you get CUDA errors
    gc.collect()
    coords = pca_project(zs)
    pca_fig = plot_embeddings_3d(coords, num_tokens, title=title+f' (PCA), epoch {epoch}', file_idx=file_idx)
    if wandb.run is not None: 
        if do_umap:
            wandb.log({f"{title} UMAP": wandb.Html(umap_fig.to_html()), f"{title} PCA": wandb.Html(pca_fig.to_html())}, step=epoch)
        else:
            wandb.log({f"{title} PCA": wandb.Html(pca_fig.to_html())}, step=epoch)
    if torch.cuda.is_available(): torch.cuda.synchronize() # cleanup again
    gc.collect()
    return pca_fig 


In [ ]:
#| export
def _subsample(data, indices, max_points):
    "Subsample data and indices together"
    perm = torch.randperm(len(data))[:max_points]
    return data[perm], indices[perm] if indices is not None else None

In [ ]:
#| export
def make_emb_viz(zs,  num_tokens, epoch=-1, model=None, title='Embeddings', max_points=8192, pmask=None, file_idx=None, do_umap=True):
    "this is the main routine, showing different groups of embeddings"
    device = zs.device
    if model is not None: model.to('cpu')
    torch.cuda.empty_cache()
    
    # CLS tokens
    cls_tokens = zs[::num_tokens]
    cls_file_idx = file_idx[::num_tokens] if file_idx is not None else None
    _make_emb_viz(cls_tokens, num_tokens, epoch=epoch, title='CLS Tokens'+title, file_idx=cls_file_idx, do_umap=do_umap)
    
    # Patches (non-CLS)
    patch_mask = torch.arange(len(zs)) % num_tokens != 0
    patch_only = zs[patch_mask]
    patch_file_idx = file_idx[patch_mask] if file_idx is not None else None
    
    if pmask is not None:
        patch_pmask = pmask[:, 1:].flatten().bool()
        print(f"Non-empty patches: {patch_pmask.sum()}/{len(patch_pmask)} ({patch_pmask.float().mean()*100:.1f}%)")
        
        # Non-empty patches
        valid_patches, valid_file_idx = patch_only[patch_pmask], (patch_file_idx[patch_pmask] if patch_file_idx is not None else None)
        rnd_patches, rnd_file_idx = _subsample(valid_patches, valid_file_idx, max_points)
        pca_fig = _make_emb_viz(rnd_patches, num_tokens, epoch=epoch, title='RND Patches'+title, file_idx=rnd_file_idx, do_umap=do_umap)
        
        # Empty patches
        empty_patches, empty_file_idx = patch_only[~patch_pmask], (patch_file_idx[~patch_pmask] if patch_file_idx is not None else None)
        rnd_empty, rnd_empty_idx = _subsample(empty_patches, empty_file_idx, max_points)
        pca_fig = _make_emb_viz(rnd_empty, num_tokens, epoch=epoch, title='RND Empty Patches'+title, do_umap=False, file_idx=rnd_empty_idx)
    
    if model is not None: model.to(device)
    return pca_fig

Testing visualization:

In [ ]:
#| eval: false
bs, num_tokens, dim = 32, 65, 256
z1 = torch.randn([bs, num_tokens, dim])
z2 = z1 +  0.1*torch.randn([bs, num_tokens, dim])
zs = torch.cat([z1, z2], dim=0).view(-1, dim)  # flatten to [64*65, 256]
#file_idx = torch.arange(bs).repeat_interleave(num_tokens)
file_idx = torch.arange(bs).repeat(2).repeat_interleave(num_tokens)

pmask = torch.ones([2*bs, num_tokens])
pmask[:, 30:] = 0  # mark roughly half the patches as empty
pca_fig = make_emb_viz(zs,  num_tokens, title='testing', pmask=pmask, file_idx=file_idx, do_umap=False) 
#pca_fig.to_html()  # Sadly, SolveIt will not render this, so you have to check this on another Jupyter Notebook system.

We're not seeing the plotly plot appear, instead I'm just seeing a bunch of raw HTML code. How can we make the plotly plot visible and solve it? This is testing code to make sure that the interactivity works and that the mouse over labeling of the file indices work. We can't use a static image and we can't use matplotlib. How do we get this working?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

**Context note:** Some earlier messages were truncated from the dialog, so I don't have the full history of what you've tried.

For Plotly in solveit, try using IPython's HTML display:

```python
from IPython.display import HTML
import plotly.express as px

fig = px.scatter(x=[1,2,3], y=[1,4,9])
HTML(fig.to_html(include_plotlyjs='cdn'))
```

This renders the plot as interactive HTML with mouse-over labels intact. Give that a try and let me know what you see!

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()